# Discord AI Agent Bot
In this notebook I will define an Agent and it will be used as an assistant in a discord channel.

- It is a Code Agent, therefore it will use code to define it's own action
- It will be able to query the web and visit web pages to reply to questions
- It can leverage another bot to set a reminder: it has to output a particular string, so it will be a sort of helper to communicate to this bot, translating natural language to a well formatted string that can be interpreted by the bot.

In [ ]:
%%capture
!pip install smolagents -U
!pip install -U discord.py

## Instantiating our agent


In [ ]:
from smolagents import CodeAgent, ToolCallingAgent, HfApiModel
from smolagents import DuckDuckGoSearchTool, VisitWebpageTool, load_tool, tool, Tool
from smolagents import TransformersModel, FinalAnswerTool, PythonInterpreterTool
import datetime
import requests
import pytz
import yaml
from huggingface_hub import login
from google.colab import userdata
login(userdata.get('HF_TOKEN'))

In [ ]:
# Function that the string outputted by the model is well formatted
import re

def check_reminder_format(input_string: str) -> bool:
    """
    Checks if the input string matches the reminder format:
    remind {nome remind}-{year}-{month}-{day}-{hour}-{minute}
    Returns True if the format is correct and the date/time components are valid.
    Raises ValueError with specific messages for different formatting errors.
    """
    pattern = r"^remind\s+([\w\s]+)\-(\d{4})\-(\d{1,2})\-(\d{1,2})\-(\d{1,2})\-(\d{1,2})$"
    match = re.match(pattern, input_string)

    if not match:
        return "Invalid reminder format. Please use: remind {nome remind}-{year}-{month}-{day}-{hour}-{minute}"

    try:
        reminder_name, year, month, day, hour, minute = match.groups()
        year, month, day, hour, minute = map(int, [year, month, day, hour, minute])
    except ValueError:
        return "Invalid date/time components. Please use integers for year, month, day, hour, and minute."

    if not (1 <= month <= 12):
        return "Invalid month. Month must be between 1 and 12."
    if not (1 <= day <= 31):
        return "Invalid day. Day must be between 1 and 31."
    if not (0 <= hour <= 23):
        return "Invalid hour. Hour must be between 0 and 23."
    if not (0 <= minute <= 59):
        return "Invalid minute. Minute must be between 0 and 59."

    return True


In [ ]:
@tool
def get_current_date_time() -> str:
    """A tool that fetches the current date and time.
    No args needed.
    Returns the current date and the time in this format: "%Y-%m-%d %H:%M:%S"
    """
    try:
        # Create timezone object
        tz = pytz.timezone("CET")
        # Get current time
        local_time = datetime.datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
        return f"{local_time}"
    except Exception as e:
        return f"Error fetching time: {str(e)}"


class SetReminder(Tool):
    name = "set_reminder"
    description = """
    An helper to set the reminder.
    It returns to you a string representing your final answer.
    The final answer returned is in this form:
    remind do the grocery-2023-5-5-12-00

    Assert that the string returned from this function starts with "remind" and then provide it as final answer with no additional character.

    Example use:
    set_reminder("do the grocery", 2023, 5, 5, 12, 00)
    """

    inputs = {
        "name": {
            "type": "string",
            "description": "The information to be reminded (e.g., 'do the grocery', 'buy milk').",
        },
        "year": {
            "type": "integer",
            "description": "The year for the reminder (e.g., 2023).",
        },
        "month": {
            "type": "integer",
            "description": "The month for the reminder (e.g., 5).",
        },
        "day": {
            "type": "integer",
            "description": "The day for the reminder (e.g., 5).",
        },
        "hour": {
            "type": "integer",
            "description": "The hour for the reminder (default: 12).",
            "nullable": True,
        },
        "minute": {
            "type": "integer",
            "description": "The minute for the reminder (default: 0).",
            "nullable": True,
        },
    }

    output_type = "string"

    def forward(self, name:str, year:int, month:int, day:int, hour:int=12, minute:int=0):
        input_string = f"remind {name}-{year}-{month}-{day}-{hour}-{minute}"
        check = check_reminder_format(input_string)
        if "Invalid" in str(check):
          return "Not correct input. " + check + "\nExample use: set_reminder(\"do the grocery\", 2023, 5, 5, 12, 00)"
        return input_string

set_reminder = SetReminder()


In [ ]:
# We're creating our CodeAgent
def create_agent():
  agent = CodeAgent(
      model = HfApiModel(),
      tools=[get_current_date_time,
             set_reminder,
             DuckDuckGoSearchTool(),
             VisitWebpageTool(),
             FinalAnswerTool()],
      max_steps=10,
      verbosity_level=1,
      grammar=None,
      planning_interval=None,
      additional_authorized_imports=['datetime'],
  )
  # print(agent.prompt_templates["system_prompt"])
  agent.visualize()
  return agent


def ask_agent(question):
  agent = create_agent()
  answer = agent.run(question)
  return answer


### Example reminder tool
In these example I show how the agent is able to output a well formatted string that trigger a bot to set a reminder.

In [ ]:
ask_agent("Ricordami domani di fare la spesa")

You are an expert assistant who can solve any task using code blobs. You will be given a task to solve as best you can.
To do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code.
To solve the task, you must plan forward to proceed in a series of steps, in a cycle of 'Thought:', 'Code:', and 'Observation:' sequences.

At each step, in the 'Thought:' sequence, you should first explain your reasoning towards solving the task and the tools that you want to use.
Then in the 'Code:' sequence, you should write the code in simple Python. The code sequence must end with '<end_code>' sequence.
During each intermediate step, you can use 'print()' to save whatever important information you will then need.
These print outputs will then appear in the 'Observation:' field, which will be available as input for the next step.
In the end you have to return a final answer using the `final_answer` tool.

Here are a few examples using n

CodeAgent | Qwen/Qwen2.5-Coder-32B-Instruct
├── ✅ Authorized imports: ['datetime']
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name                  ┃ Description                              ┃ Arguments                                ┃
    ┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ get_current_date_time │ A tool that fetches the current date and │                                          │
    │                       │ time.                                    │                                          │
    │                       │ No args needed.                          │                                          │
    │                       │ Returns the current date and the time in │                                          │
    │                       │ this format: "%Y-%m-%d %H:%M:%S"         │                                          │
    │ set_reminder          │                                          │ name (`string`): The information to be   │
    │                       │     An helper to set the reminder.       │ reminded (e.g., 'do the grocery', 'buy   │
    │                       │     It returns to you a string           │ milk').                                  │
    │                       │ representing your final answer.          │ year (`integer`): The year for the       │
    │                       │     The final answer returned is in this │ reminder (e.g., 2023).                   │
    │                       │ form:                                    │ month (`integer`): The month for the     │
    │                       │     remind do the grocery-2023-5-5-12-00 │ reminder (e.g., 5).                      │
    │                       │                                          │ day (`integer`): The day for the         │
    │                       │     Assert that the string returned from │ reminder (e.g., 5).                      │
    │                       │ this function starts with "remind" and   │ hour (`integer`): The hour for the       │
    │                       │ then provide it as final answer with no  │ reminder (default: 12).                  │
    │                       │ additional character.                    │ minute (`integer`): The minute for the   │
    │                       │                                          │ reminder (default: 0).                   │
    │                       │     Example use:                         │                                          │
    │                       │     set_reminder("do the grocery", 2023, │                                          │
    │                       │ 5, 5, 12, 00)                            │                                          │
    │                       │                                          │                                          │
    │ web_search            │ Performs a duckduckgo web search based   │ query (`string`): The search query to    │
    │                       │ on your query (think a Google search)    │ perform.                                 │
    │                       │ then returns the top search results.     │                                          │
    │ visit_webpage         │ Visits a webpage at the given url and    │ url (`string`): The url of the webpage   │
    │                       │ reads its content as a markdown string.  │ to visit.                                │
    │                       │ Use this to browse webpages.             │                                          │
    │ python_interpreter    │ This is a tool that evaluates python     │ code (`string`): The code snippet to     │
    │                       │ code. It can be used to perform          │ evaluate. All variables used in this     │
    │                       │ calculations.                            │ sn

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Ricordami domani di fare la spesa                                                                               │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  current_date_time = get_current_date_time()                                                                      
  current_year = int(current_date_time[:4])                                                                        
  current_month = int(current_date_time[5:7])                                                                      
  current_day = int(current_date_time[8:10])                                                                       
                                                                                                                   
  # Calculate tomorrow's date                                                                                      
  from datetime import datetime, timedelta                                                                         
  current_date = datetime(year=current_year, month=current_month, day=current_day)                                 
  tomorrow_date = current_date + timedelta(days=1)                                                                 
                                                                                                                   
  # Extract year, month, day for tomorrow                                                                          
  tomorrow_year = tomorrow_date.year                                                                               
  tomorrow_month = tomorrow_date.month                                                                             
  tomorrow_day = tomorrow_date.day                                                                                 
                                                                                                                   
  print(f"Current date and time: {current_date_time}")                                                             
  print(f"Tomorrow's date: {tomorrow_year}-{tomorrow_month}-{tomorrow_day}")                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  reminder = set_reminder(name="fare la spesa", year=2025, month=3, day=13, hour=12, minute=0)                     
  final_answer(reminder)                                                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: remind fare la spesa-2025-3-13-12-0

[Step 2: Duration 5.75 seconds| Input tokens: 5,711 | Output tokens: 337]

'remind fare la spesa-2025-3-13-12-0'

In [ ]:
ask_agent("Remind me ciccia sunday at 12")

You are an expert assistant who can solve any task using code blobs. You will be given a task to solve as best you can.
To do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code.
To solve the task, you must plan forward to proceed in a series of steps, in a cycle of 'Thought:', 'Code:', and 'Observation:' sequences.

At each step, in the 'Thought:' sequence, you should first explain your reasoning towards solving the task and the tools that you want to use.
Then in the 'Code:' sequence, you should write the code in simple Python. The code sequence must end with '<end_code>' sequence.
During each intermediate step, you can use 'print()' to save whatever important information you will then need.
These print outputs will then appear in the 'Observation:' field, which will be available as input for the next step.
In the end you have to return a final answer using the `final_answer` tool.

Here are a few examples using n

CodeAgent | Qwen/Qwen2.5-Coder-32B-Instruct
├── ✅ Authorized imports: ['datetime']
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name                  ┃ Description                              ┃ Arguments                                ┃
    ┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ get_current_date_time │ A tool that fetches the current date and │                                          │
    │                       │ time.                                    │                                          │
    │                       │ No args needed.                          │                                          │
    │                       │ Returns the current date and the time in │                                          │
    │                       │ this format: "%Y-%m-%d %H:%M:%S"         │                                          │
    │ set_reminder          │                                          │ name (`string`): The information to be   │
    │                       │     An helper to set the reminder.       │ reminded (e.g., 'do the grocery', 'buy   │
    │                       │     It returns to you a string           │ milk').                                  │
    │                       │ representing your final answer.          │ year (`integer`): The year for the       │
    │                       │     The final answer returned is in this │ reminder (e.g., 2023).                   │
    │                       │ form:                                    │ month (`integer`): The month for the     │
    │                       │     remind do the grocery-2023-5-5-12-00 │ reminder (e.g., 5).                      │
    │                       │                                          │ day (`integer`): The day for the         │
    │                       │     Assert that the string returned from │ reminder (e.g., 5).                      │
    │                       │ this function starts with "remind" and   │ hour (`integer`): The hour for the       │
    │                       │ then provide it as final answer with no  │ reminder (default: 12).                  │
    │                       │ additional character.                    │ minute (`integer`): The minute for the   │
    │                       │                                          │ reminder (default: 0).                   │
    │                       │     Example use:                         │                                          │
    │                       │     set_reminder("do the grocery", 2023, │                                          │
    │                       │ 5, 5, 12, 00)                            │                                          │
    │                       │                                          │                                          │
    │ web_search            │ Performs a duckduckgo web search based   │ query (`string`): The search query to    │
    │                       │ on your query (think a Google search)    │ perform.                                 │
    │                       │ then returns the top search results.     │                                          │
    │ visit_webpage         │ Visits a webpage at the given url and    │ url (`string`): The url of the webpage   │
    │                       │ reads its content as a markdown string.  │ to visit.                                │
    │                       │ Use this to browse webpages.             │                                          │
    │ python_interpreter    │ This is a tool that evaluates python     │ code (`string`): The code snippet to     │
    │                       │ code. It can be used to perform          │ evaluate. All variables used in this     │
    │                       │ calculations.                            │ sn

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Remind me ciccia sunday at 12                                                                                   │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  current_date_time = get_current_date_time()                                                                      
  print(current_date_time)                                                                                         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import datetime                                                                                                  
                                                                                                                   
  # Get the current date                                                                                           
  current_date = datetime.datetime.now().date()                                                                    
                                                                                                                   
  # Calculate the number of days until the next Sunday                                                             
  days_until_sunday = 6 - current_date.weekday()  # 0=Monday, 6=Sunday                                             
  if days_until_sunday == 0:  # If today is Sunday, set the reminder for next Sunday                               
      days_until_sunday = 7                                                                                        
                                                                                                                   
  # Calculate the date for the next Sunday                                                                         
  next_sunday = current_date + datetime.timedelta(days=days_until_sunday)                                          
                                                                                                                   
  # Extract year, month, day from the datetime object                                                              
  year = next_sunday.year                                                                                          
  month = next_sunday.month                                                                                        
  day = next_sunday.day                                                                                            
                                                                                                                   
  # Set the reminder for 12:00 on the next Sunday                                                                  
  reminder_string = set_reminder(name="ciccia", year=year, month=month, day=day, hour=12, minute=0)                
  final_answer(reminder_string)                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: remind ciccia-2025-3-16-12-0

[Step 2: Duration 14.65 seconds| Input tokens: 5,380 | Output tokens: 372]

'remind ciccia-2025-3-16-12-0'

### Example of use of the search tool

In [ ]:
ask_agent("Who are you and what can you do?")

You are an expert assistant who can solve any task using code blobs. You will be given a task to solve as best you can.
To do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code.
To solve the task, you must plan forward to proceed in a series of steps, in a cycle of 'Thought:', 'Code:', and 'Observation:' sequences.

At each step, in the 'Thought:' sequence, you should first explain your reasoning towards solving the task and the tools that you want to use.
Then in the 'Code:' sequence, you should write the code in simple Python. The code sequence must end with '<end_code>' sequence.
During each intermediate step, you can use 'print()' to save whatever important information you will then need.
These print outputs will then appear in the 'Observation:' field, which will be available as input for the next step.
In the end you have to return a final answer using the `final_answer` tool.

Here are a few examples using n

CodeAgent | Qwen/Qwen2.5-Coder-32B-Instruct
├── ✅ Authorized imports: ['datetime']
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name                  ┃ Description                              ┃ Arguments                                ┃
    ┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ get_current_date_time │ A tool that fetches the current date and │                                          │
    │                       │ time.                                    │                                          │
    │                       │ No args needed.                          │                                          │
    │                       │ Returns the current date and the time in │                                          │
    │                       │ this format: "%Y-%m-%d %H:%M:%S"         │                                          │
    │ set_reminder          │                                          │ name (`string`): The information to be   │
    │                       │     A tool that set a reminder.          │ reminded (e.g., 'do the grocery', 'buy   │
    │                       │     It returns a message communicating   │ milk').                                  │
    │                       │ if the operation succedeed.              │ year (`integer`): The year for the       │
    │                       │     Example use:                         │ reminder (e.g., 2023).                   │
    │                       │     set_reminder("do the grocery", 2023, │ month (`integer`): The month for the     │
    │                       │ 5, 5, 12, 00)                            │ reminder (e.g., 5).                      │
    │                       │                                          │ day (`integer`): The day for the         │
    │                       │                                          │ reminder (e.g., 5).                      │
    │                       │                                          │ hour (`integer`): The hour for the       │
    │                       │                                          │ reminder (default: 12).                  │
    │                       │                                          │ minute (`integer`): The minute for the   │
    │                       │                                          │ reminder (default: 0).                   │
    │ web_search            │ Performs a duckduckgo web search based   │ query (`string`): The search query to    │
    │                       │ on your query (think a Google search)    │ perform.                                 │
    │                       │ then returns the top search results.     │                                          │
    │ visit_webpage         │ Visits a webpage at the given url and    │ url (`string`): The url of the webpage   │
    │                       │ reads its content as a markdown string.  │ to visit.                                │
    │                       │ Use this to browse webpages.             │                                          │
    │ python_interpreter    │ This is a tool that evaluates python     │ code (`string`): The code snippet to     │
    │                       │ code. It can be used to perform          │ evaluate. All variables used in this     │
    │                       │ calculations.                            │ snippet must be defined in this same     │
    │                       │                                          │ snippet, else you will get an error.     │
    │                       │                                          │ This code can only import the following  │
    │                       │                                          │ python libraries: None.                  │
    │ final_answer          │ Provides a final answer to the given     │ an

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Who are you and what can you do?                                                                                │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  search_results = web_search(query="expert assistant capabilities")                                               
  print(search_results)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  answer = "I am an AI assistant designed to help with a variety of tasks, including providing information,        
  answering questions, performing calculations, and assisting with complex problem-solving, information            
  retrieval, and task automation based on the context provided."                                                   
  final_answer(answer)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: I am an AI assistant designed to help with a variety of tasks, including providing information,
answering questions, performing calculations, and assisting with complex problem-solving, information retrieval, 
and task automation based on the context provided.

[Step 2: Duration 8.86 seconds| Input tokens: 6,209 | Output tokens: 244]

'I am an AI assistant designed to help with a variety of tasks, including providing information, answering questions, performing calculations, and assisting with complex problem-solving, information retrieval, and task automation based on the context provided.'

## Executing the bot on discord

In [ ]:
import discord
import nest_asyncio # needed on google colab

intents = discord.Intents.default()
intents.message_content = True

client = discord.Client(intents=intents)
help_string = """
  This is an helper bot. Use the command
  `robot help` to get help,
  `robot <question>` to ask a question to the bot.
"""

@client.event
async def on_ready():
    print(f'We have logged in as {client.user}')

@client.event
async def on_message(message):
    if message.author == client.user:
        return

    if message.content.strip() == "robot" or message.content.strip() == "robot help":
        await message.channel.send(help_string)
        return

    if message.content.startswith('robot '):
        question = message.content[6:]
        answer = ask_agent(question)
        await message.channel.send(answer)

nest_asyncio.apply()  # Apply nest_asyncio to allow nested event loops on google colab
client.run(userdata.get('BOT_TOKEN'))
